In [ ]:
from transformers import AutoTokenizer, AutoModel

#huggingface下载的离线模型
model_name = "/root/autodl-tmp/model/bert-base-chinese"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
len(tokenizer.vocab.keys())

In [ ]:
from itertools import islice

# 使用 islice 查看词表部分内容
for key, value in islice(tokenizer.vocab.items(), 10):
    print(f"{key}: {value}")

In [ ]:
new_tokens = ["天干", "地支"]

In [ ]:
# 将集合作差结果添加到词表中
new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())

In [ ]:
new_tokens

In [ ]:
tokenizer.add_tokens(list(new_tokens))

In [ ]:
# 新增加了2个Token，词表总数由 21128 增加到 21130
len(tokenizer.vocab.keys())

In [ ]:
tokenizer.save_pretrained("/root/autodl-tmp/model/new-bert-base-chinese")

In [ ]:
model.save_pretrained("/root/autodl-tmp/model/new-bert-base-chinese", safe_serialization=False)

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertModel
import torch

model_name = "/root/autodl-tmp/model/new-bert-base-chinese"
text = "人民是[MASK]可战胜的"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

# 步骤3: 使用tokenizer处理文本
input_ids = tokenizer.encode(text, return_tensors="pt")
# 找到[MASK]标记的索引
mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

# 步骤4: 将处理后的文本输入模型进行预测
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs.logits

# 获取掩码位置的预测
mask_token_logits = predictions[0, mask_token_index, :]
top_token_ids = mask_token_logits.topk(5).indices[0].tolist()

# 步骤5: 解析模型的输出
for token_id in top_token_ids:
    predicted_token = tokenizer.decode([token_id])
    # 用预测的词替换[MASK]
    new_text = text.replace(tokenizer.mask_token, predicted_token)
    print(f"Filled text: {new_text}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of BertForMaskedLM were not initialized from the model checkpoint at /root/autodl-tmp/model/new-bert-base-chinese and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Filled text: 人民是✪可战胜的
Filled text: 人民是00㎡可战胜的
Filled text: 人民是嗆可战胜的
Filled text: 人民是6m可战胜的
Filled text: 人民是chanel可战胜的
